In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [9]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_xyzConcurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [16]:
payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [17]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [18]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [19]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [14]:
@my_decorator
def inference(thread, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
    print(f"Prompt {thread}==> {payload_data['prompt']} \n {response.text}")
    return response, payload_data['prompt'], thread
    
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    index=0
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, {**payload, 'prompt': prompt}) for srno,prompt in enumerate(prompts, start=index)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-dc2054476d00423aafb8bc342a8c5ac4","object":"text_completion","created":520479,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 4.504715 seconds 


Prompt 6==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-9df2b2120d924bfe9dc57c6fcd966902","object":"text_completion","created":520480,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapsho

Prompt 2==> Problem: Write a function in python to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime. 
 {"id":"cmpl-5e76b35f0a9f42ca9c5ea8ca2b246261","object":"text_completion","created":520479,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef is_prime(num):\n    if num <= 1:\n        return False\n    elif num <= 3:\n        return True\n    elif num % 2 == 0 or num % 3 == 0:\n        return False\n    i = 5\n    while i * i <= num:\n        if num % i == 0 or num % (i + 2) == 0:\n            return False\n        i += 6\n    return True\n\n# Test the function\nprint(is_prime(2))  # True\nprint(is_prime(4))  # False\nprint(is_prime(17))  # True\nprint(is_prime(25))  # False\nprint(is_prime(1))  # False\nprint(is_prime(0))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":37,"total_tokens":217,

Prompt 14==> Problem: Write a function in java to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-5bceb7d71dc84fc5a72ed492dd01a57b","object":"text_completion","created":520480,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple Java function that calculates the sum of the digits of a given positive integer:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        int num = 12345;\n        System.out.println(\"Sum of digits: \" + sumOfDigits(num));\n    }\n\n    public static int sumOfDigits(int num) {\n        int sum = 0;\n        while (num != 0) {\n            sum += num % 10;\n            num /= 10;\n        }\n        return sum;\n    }\n}\n```\n\nIn this function, we use a while loop to iterate through each digit of the number. In each iterat

Execution time: 16.199341 seconds 


Prompt 26==> Problem: Write a function in c++ to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-3bb324efcb2249dcb0d37f0f7cf57e8f","object":"text_completion","created":520480,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\nHere is a simple C++ function to check if two strings are anagrams:\n\n```cpp\n#include <bits/stdc++.h>\nusing namespace std;\n\nbool areAnagram(string s1, string s2) {\n    int count[256] = {0};\n    int i;\n\n    if (s1.length() != s2.length())\n        return false;\n\n    for (i = 0; s1[i] && s2[i]; i++) {\n        count[s1[i]]++;\n        count[s2[i]]--;\n    }\n\n    for (i = 0; i < 256; i++)\n    

Prompt 57==> Problem: Write a function in Golang to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-e61ce23de40d40798d55cea2273f6ebf","object":"text_completion","created":520497,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```go\npackage main\n\nimport (\n\t\"fmt\"\n\t\"strings\"\n)\n\nfunc countWords(s string) int {\n\twords := strings.Fields(s)\n\treturn len(words)\n}\n\nfunc main() {\n\ts := \"Hello, world!\"\n\tfmt.Println(countWords(s)) // Output: 2\n}\n```\n\nIn this code, the `strings.Fields` function is used to split the string into words. The `len` function is then used to count the number of words.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":38,"total_tokens":167,"completion_tokens":129}}Prompt 44==> Problem: Write a function in Ruby to calculate the sum of the digits of a given positiv

In [15]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    index=0
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, payload) for srno in range(100)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-f3c65729fed346f39fc4674a9e705cac","object":"text_completion","created":520352,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 8.118660 seconds 


Prompt 4==> Problem: Write a function in python to check if two 

Prompt 25==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-342c920d2e6f4784ba643e441a2eb74a","object":"text_completion","created":520353,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Prompt 29==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicat

Execution time: 7.728093 seconds 


Execution time: 7.605895 seconds 


Execution time: 8.052507 seconds 


Execution time: 7.545251 seconds 

Execution time: 7.825602 seconds 



Execution time: 8.250600 seconds 


Execution time: 7.640745 seconds 


Execution time: 8.151742 seconds 


Execution time: 7.583045 seconds 

Execution time: 7.704802 seconds 



Prompt 30==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-4f203ab7fd1a4bb0ac2a8424fc7c3486","object":"text_completion","created":520360,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the func


Execution time: 7.941365 seconds 

Execution time: 8.571286 seconds 

Execution time: 8.291329 seconds 



Execution time: 8.533391 seconds 


Execution time: 8.407189 seconds 




Execution time: 8.274467 seconds 


Execution time: 8.718619 seconds 


Execution time: 8.113958 seconds 


Execution time: 8.330332 seconds 


Execution time: 8.357188 seconds 

Execution time: 8.330998 seconds 



Prompt 60==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-35bde0b5907648419f139e4988d9a300","object":"text_completion","created":520367,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorte

Execution time: 8.450169 seconds 


Execution time: 8.299783 seconds 


Execution time: 8.828971 seconds 


Prompt 90==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-63c8d8e23f164b1299539102fab91171","object":"text_completion","created":520375,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 6.317636 se